In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import gc

2023-04-25 15:01:29.207743: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_train = np.load('dataset-split/batch-000.npy', allow_pickle=True)
data = pd.DataFrame(data_train, columns=['features','labels'])
data.head(15)

,features,labels
0,"[[[157, 181, 201], [156, 181, 201], [157, 182,...",0
1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
2,"[[[85, 91, 104], [87, 95, 108], [82, 94, 106],...",0
3,"[[[173, 188, 191], [172, 187, 190], [177, 192,...",0
4,"[[[236, 240, 240], [212, 216, 217], [199, 202,...",10
5,"[[[197, 199, 200], [197, 199, 200], [197, 199,...",0
6,"[[[163, 173, 183], [165, 175, 185], [166, 174,...",0
7,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
8,"[[[242, 251, 248], [242, 251, 248], [240, 249,...",0
9,"[[[196, 193, 188], [196, 193, 188], [196, 193,...",10


In [3]:
for i in data.index:
    data['features'][i] = data['features'][i].reshape(226, 226, 3)

In [4]:
np.unique(
    data['features'].iloc[i] for i in range(data['features'].size)
).size

1

In [5]:
from keras.utils import to_categorical

# Get the input data
X = np.stack(data['features'].to_numpy())

# Normalize
X = X / 255.0

# Get the labels
y = data['labels'].to_numpy()
y = y.astype(int)

# One-hot encode
y = np.vectorize(lambda x: x//5 if x>1 else x)(y)

# Subtract the minimum label value to ensure that label values start from 0
y = y - np.min(y)

# One-hot encode
y = to_categorical(y, num_classes=3)

y[:15]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [6]:
# Split the data into training and testing sets (with stratification)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [7]:
del X
del y
gc.collect()

0

In [8]:
# Define the Keras model
model_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', input_shape=X_train[0].shape),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model_cnn.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

2023-04-25 15:02:02.239077: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 15:02:02.241100: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
[type(x) for x in (X_train, y_train, X_test, y_test)]

[numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray]

In [10]:
del data
del data_train
gc.collect()

488

In [11]:
# Final preparation
X_train, y_train, X_test, y_test = tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train), tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)

# Train the model
model_cnn.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
2/2 [==============================] - 73s 35s/step - loss: 1.1037 - accuracy: 0.3333 - val_loss: 1.4907 - val_accuracy: 0.3333
Epoch 2/10
2/2 [==============================] - 72s 34s/step - loss: 1.2730 - accuracy: 0.2833 - val_loss: 1.1121 - val_accuracy: 0.2667
Epoch 3/10
2/2 [==============================] - 78s 37s/step - loss: 1.0914 - accuracy: 0.2500 - val_loss: 1.1300 - val_accuracy: 0.2667
Epoch 4/10
2/2 [==============================] - 75s 36s/step - loss: 1.0686 - accuracy: 0.4167 - val_loss: 1.1627 - val_accuracy: 0.3333
Epoch 5/10
2/2 [==============================] - 74s 37s/step - loss: 1.0825 - accuracy: 0.4833 - val_loss: 1.2947 - val_accuracy: 0.3333
Epoch 6/10
2/2 [==============================] - 72s 35s/step - loss: 0.9668 - accuracy: 0.5667 - val_loss: 1.3015 - val_accuracy: 0.2000
Epoch 7/10
2/2 [==============================] - 73s 35s/step - loss: 0.9101 - accuracy: 0.6333 - val_loss: 1.4685 - val_accuracy: 0.1333
Epoch 8/10
2/2 [===========

In [13]:
# Save the model
import os
model_cnn.save(os.path.join(os.getcwd(), 'cnn.h5'))